In [1]:
%run ../algorithmeX.ipynb

## [Polycubes](https://fr.wikipedia.org/wiki/Polycube)

In [2]:
import numpy as np
import random

def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

snsSet3Set2Colors = [(0.5529411764705883, 0.8274509803921568, 0.7803921568627451), (1.0, 1.0, 0.7019607843137254), (0.7450980392156863, 0.7294117647058823, 0.8549019607843137), (0.984313725490196, 0.5019607843137255, 0.4470588235294118), (0.5019607843137255, 0.6941176470588235, 0.8274509803921568), (0.9921568627450981, 0.7058823529411765, 0.3843137254901961), (0.7019607843137254, 0.8705882352941177, 0.4117647058823529), (0.9882352941176471, 0.803921568627451, 0.8980392156862745), (0.8509803921568627, 0.8509803921568627, 0.8509803921568627), (0.7372549019607844, 0.5019607843137255, 0.7411764705882353), (0.8, 0.9215686274509803, 0.7725490196078432), (1.0, 0.9294117647058824, 0.43529411764705883), (0.4, 0.7607843137254902, 0.6470588235294118), (0.9882352941176471, 0.5529411764705883, 0.3843137254901961), (0.5529411764705883, 0.6274509803921569, 0.796078431372549), (0.9058823529411765, 0.5411764705882353, 0.7647058823529411), (0.6509803921568628, 0.8470588235294118, 0.32941176470588235), (1.0, 0.8509803921568627, 0.1843137254901961), (0.8980392156862745, 0.7686274509803922, 0.5803921568627451), (0.7019607843137254, 0.7019607843137254, 0.7019607843137254)]
snsPastelColors = [(0.984313725490196, 0.7058823529411765, 0.6823529411764706), (0.7019607843137254, 0.803921568627451, 0.8901960784313725), (0.8, 0.9215686274509803, 0.7725490196078432), (0.8705882352941177, 0.796078431372549, 0.8941176470588236), (0.996078431372549, 0.8509803921568627, 0.6509803921568628), (1.0, 1.0, 0.8), (0.8980392156862745, 0.8470588235294118, 0.7411764705882353), (0.9921568627450981, 0.8549019607843137, 0.9254901960784314), (0.9490196078431372, 0.9490196078431372, 0.9490196078431372), (0.7019607843137254, 0.8862745098039215, 0.803921568627451), (0.9921568627450981, 0.803921568627451, 0.6745098039215687), (0.796078431372549, 0.8352941176470589, 0.9098039215686274), (0.9568627450980393, 0.792156862745098, 0.8941176470588236), (0.9019607843137255, 0.9607843137254902, 0.788235294117647), (1.0, 0.9490196078431372, 0.6823529411764706), (0.9450980392156862, 0.8862745098039215, 0.8), (0.8, 0.8, 0.8)]
snsTab20Colors = [(0.12156862745098039, 0.4666666666666667, 0.7058823529411765), (0.6823529411764706, 0.7803921568627451, 0.9098039215686274), (1.0, 0.4980392156862745, 0.054901960784313725), (1.0, 0.7333333333333333, 0.47058823529411764), (0.17254901960784313, 0.6274509803921569, 0.17254901960784313), (0.596078431372549, 0.8745098039215686, 0.5411764705882353), (0.8392156862745098, 0.15294117647058825, 0.1568627450980392), (1.0, 0.596078431372549, 0.5882352941176471), (0.5803921568627451, 0.403921568627451, 0.7411764705882353), (0.7725490196078432, 0.6901960784313725, 0.8352941176470589), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354), (0.7686274509803922, 0.611764705882353, 0.5803921568627451), (0.8901960784313725, 0.4666666666666667, 0.7607843137254902), (0.9686274509803922, 0.7137254901960784, 0.8235294117647058), (0.4980392156862745, 0.4980392156862745, 0.4980392156862745), (0.7803921568627451, 0.7803921568627451, 0.7803921568627451), (0.7372549019607844, 0.7411764705882353, 0.13333333333333333), (0.8588235294117647, 0.8588235294117647, 0.5529411764705883), (0.09019607843137255, 0.7450980392156863, 0.8117647058823529), (0.6196078431372549, 0.8549019607843137, 0.8980392156862745)]

palette = snsPastelColors

def randomColor(seed = None):
    if seed is not None:
        random.seed(str(seed))
    return random.choice(palette)

In [3]:
class PIECE3D:

    def __init__(self,nom,piece,couleur = None):
        self.nom = nom
        self.piece = piece
        self.couleur = couleur
        self.largeur = len(self.piece[0][0])
        self.profondeur = len(self.piece[0])
        self.hauteur = len(self.piece)

        self._g = lambda i: lambda c: np.rot90(c,i,(1,2))
        self._h = lambda c: np.rot90(np.rot90(c,1,(0,1)),2,(0,2))
        self._L = [(0,0,0),              (0,1,0), (0,1,1),             (0,2,1),     (0,3,0), (0,3,1),
                   (1,0,0), (1,0,1),     (1,1,0), (1,1,1),    (1,2,0), (1,2,1),     (1,3,0), (1,3,1),
                            (2,0,1),     (2,1,0), (2,1,1),    (2,2,0),              (2,3,0), (2,3,1),
                                         (3,1,0), (3,1,1),                          (3,3,0), (3,3,1),]


    def __hash__(self):
        return hash(self.piece)

    def __eq__(self,q):
        return self.piece == q.piece

    def __str__(self):
        return str((self.nom,self.piece))
 
    
    def isometriques(self):
        return list({PIECE3D(self.nom,totuple(self._g(i)(self._h(self._g(j)(self._h(self._g(k)(self.piece))))))) for i,j,k in self._L})

In [4]:
#------------------------------- POLYCUBES ----------------------------

Bent = PIECE3D( 'Bent',(((1,0),
                         (1,1)),), couleur = palette[0])
Ell = PIECE3D('Ell',(((1,0),
                      (1,0),
                      (1,1)),), couleur = palette[1])
Tee = PIECE3D('Tee',(((0,1,0),
                      (1,1,1)),), couleur = palette[2])
Skew = PIECE3D('Skew',(((0,1,1),
                        (1,1,0)),), couleur = palette[3])
Ltwist = PIECE3D('Ltwist',(((1,0),
                            (1,1)),
                           ((0,0),
                            (0,1))), couleur = palette[4])
Rtwist = PIECE3D('Rtwist',(((1,0),
                            (1,1)),
                           ((1,0),
                          (0,0))), couleur = palette[5])
Claw = PIECE3D('Claw',(((1,0),
                        (1,1)),
                       ((0,0),
                        (1,0))), couleur = palette[6])

In [5]:

class PUZZLE3D:

    def __init__(self,pieces,
                      min_i,max_i,min_j,max_j,min_k,max_k,
                      conditions = None,
                      strict = True):
        """pieces : liste de PIECE3D
        min_i,max_i,min_j,max_j,min_k,max_k : definition du volume à remplir
        conditions : triplet d'entiers -> booleen restreignant le volume
    
        1er cas  : strict = True
            Pour chacun des noms des pieces, il faut placer 
            une et une seule piece portant ce nom.
            Les elements de E sont les noms des pieces
            et les cases (triplets d'entiers) du volume.
            Chaque element de F contient un et un seul nom
            et les cases du plateau utilisees par une piece de ce nom..
        
        2eme cas : strict = False
            Pour chaque piece de pieces, on dispose,
            pour resoudre le puzzle, d'autant d'exemplaires
            que l'on veut de la piece.
            Les elements de E sont les cases du volume.
        """
        self.min_i = min_i
        self.max_i = max_i
        self.min_j = min_j
        self.max_j = max_j
        self.min_k = min_k
        self.max_k = max_k
        self.strict = strict
        self.pieces = pieces
        if not conditions:
            conditions = lambda i,j,k: True
        lignes = dict()
        nbLignes = 0
        for p in pieces:
            for u in range(min_i, max_i - p.largeur + 2):
                for v in range(min_j, max_j - p.profondeur + 2):
                    for w in range(min_k, max_k - p.hauteur + 2):
                        ligne = [p.nom] if self.strict else []
                        match = True
                        for i in range(p.largeur):
                            if match:
                                ic = i + u 
                                for j in range(p.profondeur):
                                    if match:
                                        jc = j + v
                                        for k in range(p.hauteur):
                                            kc = k + w
                                            if p.piece[k][j][i] == 1:
                                                if conditions(ic,jc,kc):
                                                    ligne.append((ic,jc,kc))
                                                else:
                                                    match = False
                                                    break
                            else:
                                break  
                        if match:
                            lignes[nbLignes] = ligne
                            nbLignes += 1
        self.lignes = lignes
        
    def solve(self):

        F = self.lignes
        return AlgorithmeX(F).solve()

    def printSolution(self,sol):     
        for l in sol: print(self.lignes[l])

    def plotSolution(self,sol,ecart=0,**kwargs):
        
        if not self.strict:
            random.seed('314')
        G = Graphics()
        for l in sol:
            e = set()
            for c in self.lignes[l]:
                if type(c) == tuple:
                    e.add(c)
                else:
                    couleur = eval(c).couleur 
            if not self.strict: couleur = randomColor()  
            couche = min([k for i,j,k in e])          
            for i,j,k in e:
                G += Polyhedron(vertices = [(i+di,j+dj,k+dk+couche*ecart) for di in [0,1] for dj in [0,1] for dk in [0,1] ]).plot(color = couleur, **kwargs)
   
        G.show(frame = False)


In [6]:
def somas(*args):
    puzzle = PUZZLE3D(Ell.isometriques() + Bent.isometriques() + Tee.isometriques() + Skew.isometriques() + \
                            Ltwist.isometriques() + Rtwist.isometriques() + Claw.isometriques(),*args)
    sols = puzzle.solve()
    s = next(sols)
    puzzle.plotSolution(s)

somas(1,3,1,3,1,3)
somas(1,4,1,3,1,3,lambda i,j,k: (i,j) not in [(1,1),(1,3),(4,2)])
somas(1,5,1,3,1,2,lambda i,j,k:(i,j,k) not in [(1,2,2),(3,2,2),(5,2,2)])

Graphics3d Object

Graphics3d Object

Graphics3d Object

In [7]:
def unePiece(piece,ecart,p,q,r):
    puzzle = PUZZLE3D(piece.isometriques(),1,p,1,q,1,r,strict = False)    
    sol = puzzle.solve()
    s = next(sol)
    puzzle.plotSolution(s, ecart = ecart)

unePiece(Ltwist,2,4,4,4)

Graphics3d Object

In [8]:
unePiece(Claw,0,4,4,2)

Graphics3d Object

In [9]:
unePiece(Skew,3,4,4,4)

Graphics3d Object

In [10]:
unePiece(Bent,2,3,3,3)

Graphics3d Object

In [13]:
# 2528 solutions essentielles

Arthur = PIECE3D('Arthur',(((0,1,0,0),
                            (1,1,1,1)),))

unePiece(Arthur,0,5,5,5)

Graphics3d Object

In [14]:
unePiece(Arthur,4,5,5,5)

Graphics3d Object